Step 1

1.

In [ ]:
import numpy as np
from scipy.stats import norm, truncnorm
import argparse
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
import time
import gc

def lookback_stratified_price(N, S0=100, K=110, r=0.05, sigma=0.3, T=1.0, m=365):
    dt = T / m
    t_grid = np.linspace(0, T, m + 1)

    # Stratification intervals for B_T ~ N(0,1)
    strata = [(-np.inf, -0.8), (-0.8, -0.4), (-0.4, -0.1), (-0.1, 0.1),
              (0.1, 0.4), (0.4, 0.8), (0.8, np.inf)]

    # Calculate probability for each stratum
    probs = [norm.cdf(b) - norm.cdf(a) for a, b in strata]
    allocations = np.round(np.array(probs) * N).astype(int)

    # Adjust to ensure total = N
    diff = N - allocations.sum()
    allocations[np.argmax(probs)] += diff

    all_payoffs = []

    for (a, b), n_paths in zip(strata, allocations):
        if n_paths == 0:
            continue

        # Sample B_T from truncated normal over stratum [a, b]
        a_std, b_std = (a - 0)/1, (b - 0)/1
        B_T_samples = truncnorm.rvs(a_std, b_std, loc=0, scale=1, size=n_paths)

        # Simulate unconditioned Brownian paths W_t
        dW = np.random.randn(n_paths, m) * np.sqrt(dt)
        W = np.hstack([np.zeros((n_paths, 1)), np.cumsum(dW, axis=1)])
        W_T = W[:, -1].reshape(-1, 1)

        # Brownian bridge: B(t) = W(t) + (t/T)(B_T - W(T))
        adjustment = (t_grid[1:].reshape(1, -1) / T) * (B_T_samples.reshape(-1, 1) - W_T)
        B_bridge = np.hstack([np.zeros((n_paths, 1)), W[:, 1:] + adjustment])

        drift = (r - 0.5 * sigma ** 2) * t_grid
        exponent = drift.reshape(1, -1) + sigma * B_bridge
        S = S0 * np.exp(exponent)
        S_max = np.max(S, axis=1)
        payoff = np.maximum(S_max - K, 0)
        all_payoffs.append(payoff)

    total_payoff = 0
    for i in range(len(strata)):
        if allocations[i] == 0:
            continue
        avg_payoff = np.mean(all_payoffs[i])
        total_payoff += probs[i] * avg_payoff

    # Discount and return price
    discounted_price = np.exp(-r * T) * total_payoff
    return discounted_price

for N in [10**4, 10**5, 10**6]:
    price = lookback_stratified_price(N)
    print(f"N = {N}: Lookback Call Price = {price:.4f}")


N = 10000: Lookback Call Price = 19.2048
N = 100000: Lookback Call Price = 18.9525
N = 1000000: Lookback Call Price = 18.9608


2.

In [ ]:
def lookback_stratified_price_equiprob(N, S0=100, K=110, r=0.05, sigma=0.3, T=1.0, m=365):
    dt = T / m
    t_grid = np.linspace(0, T, m + 1)

    # Define 7 equiprobable strata using quantiles of N(0,1)
    quantiles = np.linspace(0, 1, 8)
    z_vals = norm.ppf(quantiles)
    strata = [(z_vals[i], z_vals[i+1]) for i in range(7)]

    # Each stratum gets ~N/7 paths
    allocations = np.full(7, N // 7)
    allocations[0] += N % 7

    all_payoffs = []

    for (a, b), n_paths in zip(strata, allocations):
        if n_paths == 0:
            continue

        a_std, b_std = (a - 0)/1, (b - 0)/1
        B_T_samples = truncnorm.rvs(a_std, b_std, loc=0, scale=1, size=n_paths)

        # Simulate unconditioned Brownian paths W_t
        dW = np.random.randn(n_paths, m) * np.sqrt(dt)
        W = np.hstack([np.zeros((n_paths, 1)), np.cumsum(dW, axis=1)])
        W_T = W[:, -1].reshape(-1, 1)

        # Brownian bridge: B(t) = W(t) + (t/T)(B_T - W(T))
        adjustment = (t_grid[1:].reshape(1, -1) / T) * (B_T_samples.reshape(-1, 1) - W_T)
        B_bridge = np.hstack([np.zeros((n_paths, 1)), W[:, 1:] + adjustment])

        drift = (r - 0.5 * sigma ** 2) * t_grid
        exponent = drift.reshape(1, -1) + sigma * B_bridge
        S = S0 * np.exp(exponent)

        # Compute payoff: max(max(S) - K, 0)
        S_max = np.max(S, axis=1)
        payoff = np.maximum(S_max - K, 0)
        all_payoffs.append(payoff)

    # All strata are equiprobable → weight by 1/7
    total_payoff = sum(np.mean(p) for p in all_payoffs) / 7
    discounted_price = np.exp(-r * T) * total_payoff
    return discounted_price
print("Equiprobable Stratified Prices:")

for N in [10**4, 10**5, 10**6]:
    price = lookback_stratified_price_equiprob(N)
    print(f"N = {N}: Lookback Call Price = {price:.4f}")


Equiprobable Stratified Prices:
N = 10000: Lookback Call Price = 18.7437
N = 100000: Lookback Call Price = 19.0143
N = 1000000: Lookback Call Price = 18.9942


3.

In [ ]:
def stratified_lookback_optimal(N, N0=1000, S0=100, K=110, r=0.05, sigma=0.3, T=1.0, m=365):
    dt = T / m
    t_grid = np.linspace(0, T, m + 1)

    # Fixed strata (same as Part 1)
    strata = [(-np.inf, -0.8), (-0.8, -0.4), (-0.4, -0.1), (-0.1, 0.1),
              (0.1, 0.4), (0.4, 0.8), (0.8, np.inf)]

    # Step 1: Get probabilities for each stratum
    probs = np.array([norm.cdf(b) - norm.cdf(a) for (a, b) in strata])

    # Step 2: Pilot sample — estimate std dev in each stratum
    std_devs = []
    for (a, b) in strata:
        if norm.cdf(b) - norm.cdf(a) == 0:
            std_devs.append(0.0)
            continue

        a_std, b_std = (a - 0)/1, (b - 0)/1
        B_T_samples = truncnorm.rvs(a_std, b_std, loc=0, scale=1, size=N0)

        dW = np.random.randn(N0, m) * np.sqrt(dt)
        W = np.hstack([np.zeros((N0, 1)), np.cumsum(dW, axis=1)])
        W_T = W[:, -1].reshape(-1, 1)

        adjustment = (t_grid[1:].reshape(1, -1) / T) * (B_T_samples.reshape(-1, 1) - W_T)
        B_bridge = np.hstack([np.zeros((N0, 1)), W[:, 1:] + adjustment])

        drift = (r - 0.5 * sigma ** 2) * t_grid
        exponent = drift.reshape(1, -1) + sigma * B_bridge
        S = S0 * np.exp(exponent)

        S_max = np.max(S, axis=1)
        payoff = np.maximum(S_max - K, 0)
        std_devs.append(np.std(payoff))

    std_devs = np.array(std_devs)

    # Step 3: Compute optimal allocations
    weight = probs * std_devs
    weight_sum = weight.sum()
    allocations = np.round(N * weight / weight_sum).astype(int)
    diff = N - allocations.sum()
    if diff != 0:
        allocations[np.argmax(weight)] += diff

    # Step 4: Run main simulation with optimal allocation
    all_payoffs = []

    for (a, b), n_paths in zip(strata, allocations):
        if n_paths == 0:
            all_payoffs.append(0)
            continue

        a_std, b_std = (a - 0)/1, (b - 0)/1
        B_T_samples = truncnorm.rvs(a_std, b_std, loc=0, scale=1, size=n_paths)

        dW = np.random.randn(n_paths, m) * np.sqrt(dt)
        W = np.hstack([np.zeros((n_paths, 1)), np.cumsum(dW, axis=1)])
        W_T = W[:, -1].reshape(-1, 1)

        adjustment = (t_grid[1:].reshape(1, -1) / T) * (B_T_samples.reshape(-1, 1) - W_T)
        B_bridge = np.hstack([np.zeros((n_paths, 1)), W[:, 1:] + adjustment])

        drift = (r - 0.5 * sigma ** 2) * t_grid
        exponent = drift.reshape(1, -1) + sigma * B_bridge
        S = S0 * np.exp(exponent)

        S_max = np.max(S, axis=1)
        payoff = np.maximum(S_max - K, 0)
        avg_payoff = np.mean(payoff)
        all_payoffs.append(avg_payoff)

    total = sum(p * mean for p, mean in zip(probs, all_payoffs))
    discounted_price = np.exp(-r * T) * total
    return discounted_price

print("Optimal Allocation Stratified Prices:")
for N in [10**4, 10**5, 10**6]:
    price = stratified_lookback_optimal(N)
    print(f"N = {N}: Lookback Call Price = {price:.4f}")


Optimal Allocation Stratified Prices:
N = 10000: Lookback Call Price = 19.0360
N = 100000: Lookback Call Price = 18.9430
N = 1000000: Lookback Call Price = 18.9688


Step 2

1.

In [ ]:
def forward_start_call_mc(N, S0=100, r=0.05, sigma=0.3, T1=0.5, T2=1.0):

    # Simulate Brownian motion values
    B_T1 = np.sqrt(T1) * np.random.randn(N)
    delta_B = np.sqrt(T2 - T1) * np.random.randn(N)
    B_T2 = B_T1 + delta_B

    # Compute asset prices at T1 and T2
    drift = lambda t: (r - 0.5 * sigma**2) * t
    S_T1 = S0 * np.exp(drift(T1) + sigma * B_T1)
    S_T2 = S0 * np.exp(drift(T2) + sigma * B_T2)

    # Payoff = max(S_T2 - S_T1, 0)
    payoff = np.maximum(S_T2 - S_T1, 0)
    discounted_payoff = np.exp(-r * T2) * payoff

    price = np.mean(discounted_payoff)
    std_err = np.std(discounted_payoff) / np.sqrt(N)
    return price, std_err

print("Forward-Start Call Option Price via Standard Monte Carlo:")
for k in range(3, 9):
    N = 10**k
    price, err = forward_start_call_mc(N)
    print(f"N = {N:>7}: Price = {price:.5f}, StdErr ≈ {err:.5f}")


Forward-Start Call Option Price via Standard Monte Carlo:
N =    1000: Price = 10.32444, StdErr ≈ 0.51448
N =   10000: Price = 9.61228, StdErr ≈ 0.15289
N =  100000: Price = 9.62799, StdErr ≈ 0.04847
N = 1000000: Price = 9.65808, StdErr ≈ 0.01533
N = 10000000: Price = 9.63692, StdErr ≈ 0.00485
N = 100000000: Price = 9.63451, StdErr ≈ 0.00153


2.

In [ ]:
import numpy as np
from scipy.stats import norm

def forward_start_call_conditional_mc(N, S0=100, r=0.05, sigma=0.3, T1=0.5, T2=1.0):
    tau = T2 - T1

    # Simulate B_T1 ~ N(0, T1)
    B_T1 = np.sqrt(T1) * np.random.randn(N)

    # Simulate S_T1 from GBM
    drift_T1 = (r - 0.5 * sigma**2) * T1
    S_T1 = S0 * np.exp(drift_T1 + sigma * B_T1)

    # Black-Scholes components for normalized call: C = Phi(d1) - e^{-r tau} * Phi(d2)
    d1 = (r + 0.5 * sigma**2) * np.sqrt(tau) / sigma
    d2 = d1 - sigma * np.sqrt(tau)
    C = norm.cdf(d1) - np.exp(-r * tau) * norm.cdf(d2)

    # Compute the discounted expected payoff
    discounted_payoffs = np.exp(-r * T2) * S_T1 * C
    price = np.mean(discounted_payoffs)
    std_err = np.std(discounted_payoffs) / np.sqrt(N)

    return price, std_err

print("Forward-Start Call Option Price via Conditional Monte Carlo (Corrected):")
for k in range(3, 9):
    N = 10**k
    price, err = forward_start_call_conditional_mc(N)
    print(f"N = {N:>7}: Price = {price:.5f}, StdErr ≈ {err:.5f}")


Forward-Start Call Option Price via Conditional Monte Carlo (Corrected):
N =    1000: Price = 9.28585, StdErr ≈ 0.06546
N =   10000: Price = 9.41022, StdErr ≈ 0.02015
N =  100000: Price = 9.39498, StdErr ≈ 0.00638
N = 1000000: Price = 9.39580, StdErr ≈ 0.00202
N = 10000000: Price = 9.39741, StdErr ≈ 0.00064
N = 100000000: Price = 9.39716, StdErr ≈ 0.00020


Step 3

In [ ]:
# ----------------------------
# Parameters & Settings
# ----------------------------
r = 0.05           # risk-free rate
sigma = 0.4        # volatility
S0 = 100           # initial asset price
K = 100            # strike
T = 1.0            # maturity in years

# Discretization parameters
N_steps = 5000     # Number of time steps
dt = T / N_steps
time_grid = np.linspace(0, T, N_steps+1)

# Monte Carlo paths
N_paths = 200000   # Total number of sample paths
batch_size = 20000  # Process in batches to reduce memory usage
n_batches = N_paths // batch_size

# ----------------------------
# Black-Scholes European Put Price Function
# ----------------------------
def bs_put(S, t_to_mat, K, r, sigma):
    """Calculate the Black-Scholes European put price."""
    # Handle maturity case
    if np.isscalar(t_to_mat):
        if t_to_mat == 0:
            return np.maximum(K - S, 0)
    else:
        # Handle vectorized case
        result = np.zeros_like(S)
        maturity_mask = (t_to_mat == 0)
        result[maturity_mask] = np.maximum(K - S[maturity_mask], 0)

        # Only compute BS formula for non-maturity points
        non_maturity = ~maturity_mask
        if not np.any(non_maturity):
            return result

        S = S[non_maturity]
        t_to_mat = t_to_mat[non_maturity]

    # Black-Scholes formula
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t_to_mat) / (sigma * np.sqrt(t_to_mat))
    d2 = d1 - sigma * np.sqrt(t_to_mat)

    put_value = K * np.exp(-r * t_to_mat) * norm.cdf(-d2) - S * norm.cdf(-d1)

    if np.isscalar(t_to_mat) or np.isscalar(put_value):
        return put_value
    else:
        result[non_maturity] = put_value
        return result

# ----------------------------
# Main Function
# ----------------------------
def price_american_put_optimized():
    start_time = time.time()
    print(f"Starting simulation with {N_paths} paths in {n_batches} batches of {batch_size} paths each")

    # Precompute European put price at t=0
    P0 = bs_put(S0, T, K, r, sigma)
    print(f"European put price at t=0: {P0:.4f}")

    # Initialize accumulators for statistics
    sum_U_M_a = 0.0
    sum_U_M_b = 0.0
    sum_sq_U_M_a = 0.0
    sum_sq_U_M_b = 0.0

    sum_G_tau_a = 0.0
    sum_G_tau_b = 0.0

    sum_hedge_a = 0.0
    sum_hedge_b = 0.0

    # Process each batch
    for batch in range(n_batches):
        batch_start = time.time()
        print(f"\nProcessing batch {batch+1}/{n_batches}")

        # Set seed for reproducibility
        np.random.seed(42 + batch)

        # ----------------------------
        # 1. Simulate Stock Price Paths
        # ----------------------------
        # Generate all random numbers at once
        Z = np.random.normal(0, 1, (batch_size, N_steps))

        # Initialize paths array
        S = np.zeros((batch_size, N_steps + 1))
        S[:, 0] = S0

        # Simulate paths
        for k in range(N_steps):
            S[:, k + 1] = S[:, k] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[:, k])

        # ----------------------------
        # 2. Calculate Discounted Payoff
        # ----------------------------
        G = np.exp(-r * time_grid) * np.maximum(K - S, 0)

        # ----------------------------
        # 3. Martingale a): Discounted Risky Asset (properly defined)
        # ----------------------------
        # Key difference: Subtract constant S0 to ensure M_0 = 0 for all paths
        M_a = np.exp(-r * time_grid) * S - S0

        # Calculate upper bound for this batch
        batch_U_M_a = np.mean(np.max(G - M_a, axis=1))
        sum_U_M_a += batch_U_M_a * batch_size
        sum_sq_U_M_a += np.sum((np.max(G - M_a, axis=1))**2)

        # ----------------------------
        # 4. Martingale b): Discounted European Put Value Process
        # ----------------------------
        # Calculate European put values for all time points
        P = np.zeros((batch_size, N_steps + 1))
        for k in range(N_steps + 1):
            # Time to maturity for this time point
            t_to_mat = T - time_grid[k]
            # Compute put value for all paths at this time point
            P[:, k] = bs_put(S[:, k], t_to_mat, K, r, sigma)

        # Define martingale with M_0 = 0
        M_b = np.exp(-r * time_grid) * P - P0

        # Calculate upper bound for this batch
        batch_U_M_b = np.mean(np.max(G - M_b, axis=1))
        sum_U_M_b += batch_U_M_b * batch_size
        sum_sq_U_M_b += np.sum((np.max(G - M_b, axis=1))**2)

        # ----------------------------
        # 5. Candidate Stopping Time Analysis
        # ----------------------------
        # Function to compute stopping time and payoff using argmax approach
        def candidate_stopping_time(G, M, U_M):
            """
            Compute the candidate stopping time and corresponding payoff.
            Uses the condition: tau = inf{t: U_M + M_t <= G_t} ^ T
            """
            # Check stopping condition: U_M + M_t <= G_t
            condition = (U_M + M) <= G

            # Find first time index where condition is true
            tau_idx = np.argmax(condition, axis=1)

            # If condition never true, set to maturity
            never_exercise = ~np.any(condition, axis=1)
            tau_idx[never_exercise] = N_steps

            # Get G values at stopping times
            G_tau = G[np.arange(batch_size), tau_idx]

            return G_tau

        # Compute for both martingales
        G_tau_a = candidate_stopping_time(G, M_a, batch_U_M_a)
        G_tau_b = candidate_stopping_time(G, M_b, batch_U_M_b)

        sum_G_tau_a += np.sum(G_tau_a)
        sum_G_tau_b += np.sum(G_tau_b)

        # ----------------------------
        # 6. Hedging Suitability Analysis
        # ----------------------------
        # Evaluate the suitability of each martingale for hedging
        def hedging_suitability(G, M, U_M):
            """
            Calculate E[|sup_t (G_t - M_t) - U_M|] as a measure of hedging quality.
            """
            sup_G_minus_M = np.max(G - M, axis=1)
            return np.mean(np.abs(sup_G_minus_M - np.mean(sup_G_minus_M)))

        batch_hedge_a = hedging_suitability(G, M_a, batch_U_M_a)
        batch_hedge_b = hedging_suitability(G, M_b, batch_U_M_b)

        sum_hedge_a += batch_hedge_a * batch_size
        sum_hedge_b += batch_hedge_b * batch_size

        # Report batch results
        print(f"Batch {batch+1} completed in {time.time() - batch_start:.2f} seconds")
        print(f"  Batch U_M_a = {batch_U_M_a:.4f}")
        print(f"  Batch U_M_b = {batch_U_M_b:.4f}")

        # Clean up memory
        del S, G, M_a, M_b, P, Z
        gc.collect()

    # ----------------------------
    # 7. Calculate Final Statistics
    # ----------------------------
    # American put price estimates
    U_M_a = sum_U_M_a / N_paths
    U_M_b = sum_U_M_b / N_paths

    # Variance and standard errors
    var_U_M_a = (sum_sq_U_M_a / N_paths) - (U_M_a ** 2)
    var_U_M_b = (sum_sq_U_M_b / N_paths) - (U_M_b ** 2)
    std_error_a = np.sqrt(var_U_M_a / N_paths)
    std_error_b = np.sqrt(var_U_M_b / N_paths)

    # Expected payoff at stopping times
    E_G_tau_a = sum_G_tau_a / N_paths
    E_G_tau_b = sum_G_tau_b / N_paths

    # Hedging suitability measures
    hedge_a = sum_hedge_a / N_paths
    hedge_b = sum_hedge_b / N_paths

    # ----------------------------
    # 8. Print Results
    # ----------------------------
    print("\n===== RESULTS =====")
    print("--- Dual Estimator Results ---")
    print(f"Candidate (a): Discounted asset martingale")
    print(f"  Upper bound U_M_a = {U_M_a:.4f} ± {1.96*std_error_a:.4f}")
    print(f"Candidate (b): Discounted European put price martingale")
    print(f"  Upper bound U_M_b = {U_M_b:.4f} ± {1.96*std_error_b:.4f}")

    print("\n--- Stopping Time Analysis ---")
    print(f"Candidate (a): Discounted asset martingale")
    print(f"  E[G(tau)] = {E_G_tau_a:.4f}")
    print(f"  Interval [E[G(tau)], U_M] = [{E_G_tau_a:.4f}, {U_M_a:.4f}]")
    print(f"  Interval width = {U_M_a - E_G_tau_a:.4f}")

    print(f"Candidate (b): Discounted European put martingale")
    print(f"  E[G(tau)] = {E_G_tau_b:.4f}")
    print(f"  Interval [E[G(tau)], U_M] = [{E_G_tau_b:.4f}, {U_M_b:.4f}]")
    print(f"  Interval width = {U_M_b - E_G_tau_b:.4f}")
    print("-> A narrower interval [E[G(tau)], U_M] indicates a near optimal exercise strategy.")

    print("\n--- Hedging Error Measure ---")
    print(f"Candidate (a): Discounted asset martingale")
    print(f"  E[|sup_t (G-M) - U_M|] = {hedge_a:.4f}")
    print(f"Candidate (b): Discounted European put martingale")
    print(f"  E[|sup_t (G-M) - U_M|] = {hedge_b:.4f}")

    print("\n--- Summary ---")
    if hedge_a < hedge_b:
        print("For hedging, candidate (a) seems to perform better.")
    else:
        print("For hedging, candidate (b) (the European put process) performs better.")

    # European price for comparison
    print(f"\nEuropean put price (Black-Scholes): {P0:.4f}")
    print(f"Early exercise premium (a): {U_M_a - P0:.4f}")
    print(f"Early exercise premium (b): {U_M_b - P0:.4f}")

    # Create comparison with waiting until maturity
    european_premium = P0 - (np.exp(-r * T) * bs_put(S0, 0, K, r, sigma))
    print(f"\nWaiting until maturity vs early exercise:")
    if E_G_tau_a > P0:
        print(f"For martingale (a): Exercising at tau is better by {E_G_tau_a - P0:.4f}")
    else:
        print(f"For martingale (a): Waiting until maturity is better by {P0 - E_G_tau_a:.4f}")

    if E_G_tau_b > P0:
        print(f"For martingale (b): Exercising at tau is better by {E_G_tau_b - P0:.4f}")
    else:
        print(f"For martingale (b): Waiting until maturity is better by {P0 - E_G_tau_b:.4f}")

    print(f"\nTotal execution time: {time.time() - start_time:.2f} seconds")

# ----------------------------
# Run the simulation
# ----------------------------

price_american_put_optimized()

Starting simulation with 200000 paths in 10 batches of 20000 paths each
European put price at t=0: 13.1459

Processing batch 1/10
Batch 1 completed in 35.30 seconds
  Batch U_M_a = 53.4525
  Batch U_M_b = 13.8642

Processing batch 2/10
Batch 2 completed in 32.49 seconds
  Batch U_M_a = 53.2383
  Batch U_M_b = 13.8534

Processing batch 3/10
Batch 3 completed in 37.38 seconds
  Batch U_M_a = 53.3891
  Batch U_M_b = 13.8611

Processing batch 4/10
Batch 4 completed in 33.67 seconds
  Batch U_M_a = 53.3855
  Batch U_M_b = 13.8597

Processing batch 5/10
Batch 5 completed in 32.88 seconds
  Batch U_M_a = 52.8216
  Batch U_M_b = 13.8483

Processing batch 6/10
Batch 6 completed in 34.18 seconds
  Batch U_M_a = 52.8430
  Batch U_M_b = 13.8519

Processing batch 7/10
Batch 7 completed in 34.90 seconds
  Batch U_M_a = 53.1063
  Batch U_M_b = 13.8551

Processing batch 8/10
Batch 8 completed in 34.49 seconds
  Batch U_M_a = 53.3990
  Batch U_M_b = 13.8577

Processing batch 9/10
Batch 9 completed in 3

Problem 4

M = 7

In [ ]:
def problem4_bermudan_basket(N = 10000000):

    print("\n===== PROBLEM #4: BERMUDAN BASKET OPTION =====\n")

    # Parameters
    # Parameters
    r = 0.05
    sigma = 0.3
    S0 = 100
    K = 100
    T = 1.0
    nSteps = 12
    dt = T / nSteps

    # Correlation settings: each pair has rho=0.2
    rho = 0.2
    corr_matrix = np.array([[1.0, rho, rho],
                            [rho, 1.0, rho],
                            [rho, rho, 1.0]])
    # Cholesky factorization for generating correlated normals
    L = np.linalg.cholesky(corr_matrix)

    # Pre-calculate constants for simulation
    drift = (r - 0.5 * sigma**2) * dt
    diff = sigma * np.sqrt(dt)

    # Simulate asset price paths for 3 assets.
    # S is a (3, N, nSteps+1) array with each asset's path.
    S = np.empty((3, N, nSteps+1))
    S[:, :, 0] = S0

    # Loop over time steps to simulate the GBM paths
    for t in range(1, nSteps+1):
        # Generate independent N(0,1) variates for 3 assets and N paths (shape (3, N))
        z = np.random.randn(3, N)
        # Impose the correlation via the Cholesky factor
        correlated_z = L @ z
        # Update asset prices using the discretized GBM formula
        S[:, :, t] = S[:, :, t-1] * np.exp(drift + diff * correlated_z)

    # Compute basket values as the sum of asset prices (shape (N, nSteps+1))
    basket = np.sum(S, axis=0)

    # Initialize cash flows at maturity: payoff = (basket - K)^+
    cash_flow = np.maximum(basket[:, -1] - K, 0)
    # Optional: keep track of exercise times (here not used further)
    exercise_time = np.full(N, nSteps)

    # Backward induction: work backwards from time step nSteps-1 to 1
    for t in range(nSteps-1, 0, -1):
        # Immediate payoff at time t for exercising the option
        immediate_payoff = np.maximum(basket[:, t] - K, 0)
        # Identify in-the-money paths at time t
        itm = immediate_payoff > 0
        # Discount the future cash flow back one period for in-the-money paths
        disc_cash_flow = cash_flow[itm] * np.exp(-r * dt)

        # Only perform regression if at least one path is in the money.
        if np.sum(itm) > 0:
            # For in-the-money paths, extract the individual asset prices at time t.
            S1_itm = S[0, itm, t]
            S2_itm = S[1, itm, t]
            S3_itm = S[2, itm, t]

            # Build the expanded basis for the regression on in-the-money paths.
            # Basis functions: constant, individual asset prices, squares, and interaction terms.
            A_itm = np.column_stack([
                np.ones_like(S1_itm),
                S1_itm,
                S2_itm,
                S3_itm,
                S1_itm**2,
                S2_itm**2,
                S3_itm**2
            ])

            # Regression: solve for beta in the linear model
            #   continuation_value ≈ A_itm @ beta
            beta, _, _, _ = np.linalg.lstsq(A_itm, disc_cash_flow, rcond=None)

            # Now, for all paths at time t, compute the same basis functions using the individual asset prices.
            S1_all = S[0, :, t]
            S2_all = S[1, :, t]
            S3_all = S[2, :, t]

            A_all = np.column_stack([
                np.ones_like(S1_all),
                S1_all,
                S2_all,
                S3_all,
                S1_all**2,
                S2_all**2,
                S3_all**2
            ])

            # Estimated continuation value for all paths based on the regression
            cont_value = A_all @ beta

            # For in-the-money paths, decide whether to exercise:
            # If the immediate payoff exceeds the estimated continuation value, choose to exercise.
            exercise = (immediate_payoff > cont_value)
        else:
            # If no path is in the money, then there is no exercise decision,
            # simply discount the cash flow.
            exercise = np.full(N, False)

        # Update cash flows:
        # If exercise occurs, set the cash flow to the immediate payoff; otherwise, discount
        # the existing cash flow from the next time step.
        new_cash_flow = cash_flow * np.exp(-r * dt)
        new_cash_flow[exercise] = immediate_payoff[exercise]

        cash_flow = new_cash_flow.copy()

    # Finally, discount cash flows from t=0.
    price_estimate = np.mean(cash_flow * np.exp(-r * dt))
    sample_variance = np.var(cash_flow * np.exp(-r * dt)) / N

    # Display results
    print("\n===== RESULTS FOR PROBLEM #4 =====")
    print(f"Bermudan basket option price: {price_estimate:.6f}")
    print(f"Standard error: {(sample_variance**0.5):.6f}")
    print(f"Sample Variance: {(sample_variance):.6f}")
    print(f"95% confidence interval: [{price_estimate - 1.96*(sample_variance**0.5):.6f}, {price_estimate + 1.96*(sample_variance**0.5):.6f}]")
    print(f"Number of paths: {N}")

    return {
        'option_price': price_estimate,
        'sample_variance': sample_variance,
        'std_error': sample_variance**0.5
    }

problem4_bermudan_basket(10000000)


===== PROBLEM #4: BERMUDAN BASKET OPTION =====


===== RESULTS FOR PROBLEM #4 =====
Bermudan basket option price: 204.861147
Standard error: 0.019778
Sample Variance: 0.000391
95% confidence interval: [204.822383, 204.899911]
Number of paths: 10000000


{'option_price': np.float64(204.8611468857973),
 'sample_variance': np.float64(0.0003911570053190503),
 'std_error': np.float64(0.01977768958496038)}

M = 10

In [ ]:
def problem4_bermudan_basket(N = 10000000):

    print("\n===== PROBLEM #4: BERMUDAN BASKET OPTION =====\n")

    # Parameters
    # Parameters
    r = 0.05
    sigma = 0.3
    S0 = 100
    K = 100
    T = 1.0
    nSteps = 12
    dt = T / nSteps

    # Correlation settings: each pair has rho=0.2
    rho = 0.2
    corr_matrix = np.array([[1.0, rho, rho],
                            [rho, 1.0, rho],
                            [rho, rho, 1.0]])
    # Cholesky factorization for generating correlated normals
    L = np.linalg.cholesky(corr_matrix)

    # Pre-calculate constants for simulation
    drift = (r - 0.5 * sigma**2) * dt
    diff = sigma * np.sqrt(dt)

    # Simulate asset price paths for 3 assets.
    # S is a (3, N, nSteps+1) array with each asset's path.
    S = np.empty((3, N, nSteps+1))
    S[:, :, 0] = S0

    # Loop over time steps to simulate the GBM paths
    for t in range(1, nSteps+1):
        # Generate independent N(0,1) variates for 3 assets and N paths (shape (3, N))
        z = np.random.randn(3, N)
        # Impose the correlation via the Cholesky factor
        correlated_z = L @ z
        # Update asset prices using the discretized GBM formula
        S[:, :, t] = S[:, :, t-1] * np.exp(drift + diff * correlated_z)

    # Compute basket values as the sum of asset prices (shape (N, nSteps+1))
    basket = np.sum(S, axis=0)

    # Initialize cash flows at maturity: payoff = (basket - K)^+
    cash_flow = np.maximum(basket[:, -1] - K, 0)
    # Optional: keep track of exercise times (here not used further)
    exercise_time = np.full(N, nSteps)

    # Backward induction: work backwards from time step nSteps-1 to 1
    for t in range(nSteps-1, 0, -1):
        # Immediate payoff at time t for exercising the option
        immediate_payoff = np.maximum(basket[:, t] - K, 0)
        # Identify in-the-money paths at time t
        itm = immediate_payoff > 0
        # Discount the future cash flow back one period for in-the-money paths
        disc_cash_flow = cash_flow[itm] * np.exp(-r * dt)

        # Only perform regression if at least one path is in the money.
        if np.sum(itm) > 0:
            # For in-the-money paths, extract the individual asset prices at time t.
            S1_itm = S[0, itm, t]
            S2_itm = S[1, itm, t]
            S3_itm = S[2, itm, t]

            # Build the expanded basis for the regression on in-the-money paths.
            # Basis functions: constant, individual asset prices, squares, and interaction terms.
            A_itm = np.column_stack([
                np.ones_like(S1_itm),
                S1_itm,
                S2_itm,
                S3_itm,
                S1_itm**2,
                S2_itm**2,
                S3_itm**2,
                S1_itm**3,
                S2_itm**3,
                S3_itm**3
            ])

            # Regression: solve for beta in the linear model
            #   continuation_value ≈ A_itm @ beta
            beta, _, _, _ = np.linalg.lstsq(A_itm, disc_cash_flow, rcond=None)

            # Now, for all paths at time t, compute the same basis functions using the individual asset prices.
            S1_all = S[0, :, t]
            S2_all = S[1, :, t]
            S3_all = S[2, :, t]

            A_all = np.column_stack([
                np.ones_like(S1_all),
                S1_all,
                S2_all,
                S3_all,
                S1_all**2,
                S2_all**2,
                S3_all**2,
                S1_all**3,
                S2_all**3,
                S3_all**3
            ])

            # Estimated continuation value for all paths based on the regression
            cont_value = A_all @ beta

            # For in-the-money paths, decide whether to exercise:
            # If the immediate payoff exceeds the estimated continuation value, choose to exercise.
            exercise = (immediate_payoff > cont_value)
        else:
            # If no path is in the money, then there is no exercise decision,
            # simply discount the cash flow.
            exercise = np.full(N, False)

        # Update cash flows:
        # If exercise occurs, set the cash flow to the immediate payoff; otherwise, discount
        # the existing cash flow from the next time step.
        new_cash_flow = cash_flow * np.exp(-r * dt)
        new_cash_flow[exercise] = immediate_payoff[exercise]

        cash_flow = new_cash_flow.copy()

    # Finally, discount cash flows from t=0.
    price_estimate = np.mean(cash_flow * np.exp(-r * dt))
    sample_variance = np.var(cash_flow * np.exp(-r * dt)) / N

    # Display results
    print("\n===== RESULTS FOR PROBLEM #4 =====")
    print(f"Bermudan basket option price: {price_estimate:.6f}")
    print(f"Standard error: {(sample_variance**0.5):.6f}")
    print(f"Sample Variance: {(sample_variance):.6f}")
    print(f"95% confidence interval: [{price_estimate - 1.96*(sample_variance**0.5):.6f}, {price_estimate + 1.96*(sample_variance**0.5):.6f}]")
    print(f"Number of paths: {N}")

    return {
        'option_price': price_estimate,
        'sample_variance': sample_variance,
        'std_error': sample_variance**0.5
    }

problem4_bermudan_basket(10000000)


===== PROBLEM #4: BERMUDAN BASKET OPTION =====


===== RESULTS FOR PROBLEM #4 =====
Bermudan basket option price: 204.873753
Standard error: 0.019793
Sample Variance: 0.000392
95% confidence interval: [204.834960, 204.912546]
Number of paths: 10000000


{'option_price': np.float64(204.87375287015095),
 'sample_variance': np.float64(0.00039174395925836166),
 'std_error': np.float64(0.019792522811869176)}